In [1]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd
import numpy as np

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [2]:
# Inputs
new_parcels = r"E:\Projects\REMM-Input-Data-Prep-2019\Parcels\2020-WFRC\Outputs\remm_base_year_2019.gdb\new_parcels_old_parcels_sj"
new_parcels = pd.DataFrame.spatial.from_featureclass(new_parcels)
old_parcels = pd.read_csv(r"E:\REMM\Base_Year_Data_2015\parcels.csv")

In [3]:
new_parcels.columns
new_parcels = new_parcels[['parcel_id_REMM', 'CO_NAME', 'COUNTY_ID', 'PARCEL_ID',
       'TOTAL_MKT_VALUE', 'land_value', 'residential_units', 'building_sqft',
       'FLOORS_CNT', 'year_built', 'EFFBUILT_YR', 'IS_OUG', 'max_dua',
       'max_far', 'max_height', 'type1', 'type2', 'type3', 'type4', 'type5',
       'type6', 'type7', 'type8', 'agriculture', 'basebldg', 'NoBuild',
       'redev_friction', 'building_type_id', 'x', 'y', 'parcel_acres', 'Split',
       'Split_Factor', 'parcel_id_REMM_old', 'building_type',
       'residential_price', 'non_residential_price', 'non_residential_sqft',
       'TAZID_832', 'TAZID_900', 'Old_PID']].copy()

In [4]:
old_parcels = old_parcels[['parcel_id_REMM', 'parent_parcel','elevation', 'fwy_exit', 'airport',
       'rail_depot', 'stream', 'trail', 'university', 'shape_area',
       'volume_one_way', 'volume_two_way', 'airport_distance', 'fwy_exit_dist',
       'raildepot_dist', 'university_dist', 'trail_dist', 'stream_dist',
       'train_station', 'rail_stn_dist', 'bus_rte_dist', 'bus_stop',
       'bus_stop_dist', 'volume_two_way_nofwy', 'distsml_id', 'distmed_id',
       'distlrg_id','zonal_ppa']].copy()

old_parcels.rename({'parcel_id_REMM':'Old_PID'},  axis=1, inplace=True)

In [28]:
old_parcels.columns

Index(['Old_PID', 'elevation', 'fwy_exit', 'airport', 'rail_depot', 'stream',
       'trail', 'university', 'shape_area', 'volume_one_way', 'volume_two_way',
       'airport_distance', 'fwy_exit_dist', 'raildepot_dist',
       'university_dist', 'trail_dist', 'stream_dist', 'train_station',
       'rail_stn_dist', 'bus_rte_dist', 'bus_stop', 'bus_stop_dist',
       'volume_two_way_nofwy', 'distsml_id', 'distmed_id', 'distlrg_id',
       'zonal_ppa'],
      dtype='object')

In [5]:
parcels = new_parcels.merge(old_parcels, left_on='Old_PID', right_on='Old_PID', how='left')
parcel_eq = parcels[['parcel_id_REMM', 'x', 'y', 'Old_PID', 'parent_parcel']]
parcel_eq.columns = ['parcel_id', 'x', 'y', 'Old_PID', 'Old_parent_parcel']

In [6]:
parcels.to_csv(r"E:\REMM\Base_Year_Data_2019\parcels.csv", index = False)
parcel_eq.to_csv(r"E:\REMM\Base_Year_Data_2019\parcel_eq.csv", index = False)

## update distance to transit stops

In [22]:
from simpledbf import Dbf5
import pandas as pd
import arcpy

d2ts2015 = Dbf5(r"E:\Projects\REMM2_For_Python3_2019\TDM\_TDMv8.3_REMM\REMM\REMM_2015\6_REMM\parceldistance2transitstops.dbf").to_dataframe()
d2ts2022 = Dbf5(r"E:\Projects\REMM2_For_Python3_2019\TDM\_TDMv8.3_REMM\REMM\REMM_2022\6_REMM\parceldistance2transitstops.dbf").to_dataframe()
d2ts2027 =Dbf5(r"E:\Projects\REMM2_For_Python3_2019\TDM\_TDMv8.3_REMM\REMM\REMM_2027\6_REMM\parceldistance2transitstops.dbf").to_dataframe()
d2ts2035 = Dbf5(r"E:\Projects\REMM2_For_Python3_2019\TDM\_TDMv8.3_REMM\REMM\REMM_2035\6_REMM\parceldistance2transitstops.dbf").to_dataframe()
d2ts2045 = Dbf5(r"E:\Projects\REMM2_For_Python3_2019\TDM\_TDMv8.3_REMM\REMM\REMM_2045\6_REMM\parceldistance2transitstops.dbf").to_dataframe()

parcel_eq = pd.read_csv(r"E:\REMM\Base_Year_Data_2019\parcel_eq.csv")
parcel_eq.columns = ['parcel_id_new', 'x', 'y', 'Old_PID', 'Old_parent_parcel']

In [23]:
d2ts2015 = parcel_eq.merge(d2ts2015, left_on='Old_parent_parcel', right_on='parcel_id', how='left')
d2ts2015 = d2ts2015[['parcel_id_new','dist2busst','dist2rails']]
d2ts2015.columns = ['parcel_id','dist2busst','dist2rails']
d2ts2015.fillna(0, inplace=True)
d2ts2015.to_csv(r"E:\REMM\Base_Year_Data_2019\parceldistance2transitstops_2015.csv", index = False)
arcpy.TableToTable_conversion(r"E:\REMM\Base_Year_Data_2019\parceldistance2transitstops_2015.csv", r"E:\REMM\Base_Year_Data_2019", 'parceldistance2transitstops_2015.dbf')

d2ts2022 = parcel_eq.merge(d2ts2022, left_on='Old_parent_parcel', right_on='parcel_id', how='left')
d2ts2022 = d2ts2022[['parcel_id_new','dist2busst','dist2rails']]
d2ts2022.columns = ['parcel_id','dist2busst','dist2rails']
d2ts2022.fillna(0, inplace=True)
d2ts2022.to_csv(r"E:\REMM\Base_Year_Data_2019\parceldistance2transitstops_2022.csv", index = False)
arcpy.TableToTable_conversion(r"E:\REMM\Base_Year_Data_2019\parceldistance2transitstops_2022.csv", r"E:\REMM\Base_Year_Data_2019", 'parceldistance2transitstops_2022.dbf')

d2ts2027 = parcel_eq.merge(d2ts2027, left_on='Old_parent_parcel', right_on='parcel_id', how='left')
d2ts2027 = d2ts2027[['parcel_id_new','dist2busst','dist2rails']]
d2ts2027.columns = ['parcel_id','dist2busst','dist2rails']
d2ts2027.fillna(0, inplace=True)
d2ts2027.to_csv(r"E:\REMM\Base_Year_Data_2019\parceldistance2transitstops_2027.csv", index = False)
arcpy.TableToTable_conversion(r"E:\REMM\Base_Year_Data_2019\parceldistance2transitstops_2027.csv", r"E:\REMM\Base_Year_Data_2019", 'parceldistance2transitstops_2027.dbf')

d2ts2035 = parcel_eq.merge(d2ts2035, left_on='Old_parent_parcel', right_on='parcel_id', how='left')
d2ts2035 = d2ts2035[['parcel_id_new','dist2busst','dist2rails']]
d2ts2035.columns = ['parcel_id','dist2busst','dist2rails']
d2ts2035.fillna(0, inplace=True)
d2ts2035.to_csv(r"E:\REMM\Base_Year_Data_2019\parceldistance2transitstops_2035.csv", index = False)
arcpy.TableToTable_conversion(r"E:\REMM\Base_Year_Data_2019\parceldistance2transitstops_2035.csv", r"E:\REMM\Base_Year_Data_2019", 'parceldistance2transitstops_2035.dbf')

d2ts2045 = parcel_eq.merge(d2ts2045, left_on='Old_parent_parcel', right_on='parcel_id', how='left')
d2ts2045 = d2ts2045[['parcel_id_new','dist2busst','dist2rails']]
d2ts2045.columns = ['parcel_id','dist2busst','dist2rails']
d2ts2045.fillna(0, inplace=True)
d2ts2045.to_csv(r"E:\REMM\Base_Year_Data_2019\parceldistance2transitstops_2045.csv", index = False)
arcpy.TableToTable_conversion(r"E:\REMM\Base_Year_Data_2019\parceldistance2transitstops_2045.csv", r"E:\REMM\Base_Year_Data_2019", 'parceldistance2transitstops_2045.dbf')

<Result 'E:\\REMM\\Base_Year_Data_2019\\parceldistance2transitstops_2045.dbf'>

In [15]:
d2ts2015

,parcel_id,dist2busst,dist2rails
0,NaN,177,5898
1,NaN,956,12801
2,NaN,866,15629
3,NaN,1783,13923
4,NaN,727,10875
...,...,...,...
1116286,28655.0,2581,113025
1116287,35251.0,2581,113025
1116288,NaN,7039,77896
1116289,NaN,8288,78209


In [9]:
d2ts2015

,parcel_id,dist2busst,dist2rails,parcel_id_new,x,y,Old_PID,Old_parent_parcel
0,1,177,5898,NaN,NaN,NaN,NaN,NaN
1,2,956,12801,NaN,NaN,NaN,NaN,NaN
2,3,866,15629,NaN,NaN,NaN,NaN,NaN
3,4,1783,13923,NaN,NaN,NaN,NaN,NaN
4,5,727,10875,456936.0,1.519511e+06,7.431385e+06,5.0,5.0
...,...,...,...,...,...,...,...,...
880593,753412,2599,113037,502997.0,1.574133e+06,7.271296e+06,753412.0,753412.0
880594,753413,2581,113025,503007.0,1.574193e+06,7.271340e+06,753413.0,753413.0
880595,753414,7039,77896,NaN,NaN,NaN,NaN,NaN
880596,753415,8288,78209,561309.0,1.515917e+06,7.300627e+06,753415.0,753415.0
